In [2]:
from drone_detector.utils import *
from drone_detector.imports import *
import os
from drone_detector.metrics import *
import warnings
warnings.filterwarnings("ignore")

In [3]:
def intersection_over_area(poly_1, poly_2):
    "Proportion of the overlap of poly_1 and poly_2 of the area of poly_1"
    area_intersection = poly_1.intersection(poly_2).area
    return area_intersection / poly_1.area

In [4]:
def merge_polys(gdf, area_threshold=0.2):
    polys_to_merge = {}
    polys_to_keep = []
    gdf = gdf.copy()
    gdf.reset_index(drop=True, inplace=True)
    for ann in tqdm(gdf.itertuples()):
        overlaps = gdf[gdf.geometry.intersects(ann.geometry)].copy()
        if len(overlaps) == 1: 
            polys_to_keep.append(ann.Index)
            continue
        overlaps = overlaps[overlaps.index != ann.Index]
        overlaps['pcts'] = overlaps.apply(lambda row: intersection_over_area(ann.geometry, row.geometry), axis=1)
        to_merge = overlaps[overlaps.pcts > area_threshold] # merge if intersection over area is larger than thr
        if len(to_merge) > 0:
            polys_to_merge[ann.Index] = [i for i in to_merge.index]
        else:
            polys_to_keep.append(ann.Index)
            
    merge_pairs = []
    for key, val in polys_to_merge.items():
        other_vals = ([polys_to_merge[k] for k in val if k in polys_to_merge.keys()])
        other_vals = [i for s in other_vals for i in s]
        merge_vals = [val]
        merge_vals.append(list(set([key] + other_vals)))
        merge_vals = sorted([v for s in merge_vals for v in s])
        merge_pairs.append(tuple(merge_vals))
        
    new_polys = {'label': [], 'score': [], 'geometry': []}
    for ixs in list(set(merge_pairs)):
        tempdf = gdf.iloc[list(ixs)]
        label = tempdf.label.unique()[0]
        scores = tempdf.score.unique()
        avg_score = np.mean(scores)
        geoms = tempdf.geometry
        new_polys['label'].append(label)
        new_polys['score'].append(avg_score)
        new_polys['geometry'].append(shapely.ops.unary_union(geoms))
    final_gdf = pd.concat([gdf.iloc[polys_to_keep], gpd.GeoDataFrame(new_polys, crs=gdf.crs)])
    final_gdf['geometry'] = final_gdf.apply(lambda row: fix_multipolys(row.geometry) 
                                            if row.geometry.type == 'MultiPolygon' 
                                            else shapely.geometry.Polygon(row.geometry.exterior), axis=1)
    final_gdf.drop_duplicates(subset=['geometry'], inplace=True)
    return final_gdf

# Virtual plot level object detection results

# Hiidenportti test set

As Hiidenportti test set is so small, we can run predictions here if needed.

## No overlap, no post-processing

In [5]:
from drone_detector.engines.detectron2.predict import predict_instance_masks
raw_path = Path('../../data/raw/hiidenportti/virtual_plots/buffered_test/images')
test_rasters = [raw_path/f for f in os.listdir(raw_path) if f.endswith('tif')]

In [6]:
pred_outpath = Path('../results/hp_unprocessed/')
if not os.path.exists(pred_outpath):
    shutil.copytree('../results/template_folder/', pred_outpath)

In [ ]:
for t in test_rasters:
    outfile_name = pred_outpath/f'raw_preds/{str(t).split("/")[-1][:-4]}.geojson'
    predict_instance_masks(path_to_model_config='../models/hiidenportti/mask_rcnn_X_101_32x8d_FPN_3x/config.yaml', 
                           path_to_image=str(t),
                           outfile=str(outfile_name),
                           processing_dir='temp',
                           tile_size=512,
                           tile_overlap=0,
                           smooth_preds=False,
                           use_tta=True,
                           coco_set='../../data/processed/hiidenportti/hiidenportti_valid.json',
                           postproc_results=False)

In [10]:
raw_res_path = Path('../results/hp_unprocessed/')
truth_shps = sorted([raw_res_path/'vector_tiles'/f for f in os.listdir(raw_res_path/'vector_tiles')])
raw_shps = sorted([raw_res_path/'raw_preds'/f for f in os.listdir(raw_res_path/'raw_preds')])
rasters = sorted([raw_res_path/'raster_tiles'/f for f in os.listdir(raw_res_path/'raster_tiles')])

"Raw" predictions are modified as such:
1. Invalid polygons are fixed to be valid polygons. MultiPolygon masks are replaced with the largest single polygon of the multipoly.
2. Extent is clipped to be same as the corresponding ground truth data
3. Label numbering is adjusted
4. Polygons with area less than 16² pixels are discarded

In [12]:
for p, t in zip(raw_shps, truth_shps):
    temp_pred = gpd.read_file(p)
    temp_truth = gpd.read_file(t)
    temp_pred['geometry'] = temp_pred.apply(lambda row: fix_multipolys(row.geometry) 
                                            if row.geometry.type == 'MultiPolygon' 
                                            else shapely.geometry.Polygon(row.geometry.exterior), axis=1)
    temp_pred['label'] += 1
    temp_pred = gpd.clip(temp_pred, box(*temp_truth.total_bounds))
    temp_pred = temp_pred[temp_pred.geometry.area > 16*0.04**2]
    temp_pred.to_file(raw_res_path/'predicted_vectors'/p.name)

In [13]:
pred_shps = sorted([raw_res_path/'predicted_vectors'/f for f in os.listdir(raw_res_path/'predicted_vectors')])

In [14]:
truths = None
preds = None

for p, t in zip(pred_shps, truth_shps):
    temp_pred = gpd.read_file(p)
    temp_truth = gpd.read_file(t)
    if truths is None:
        truths = temp_truth
        preds = temp_pred
    else:
        truths = pd.concat((truths, temp_truth))
        preds = pd.concat((preds, temp_pred))

In [15]:
preds['layer'] = preds.apply(lambda row: 'groundwood' if row.label == 2 else 'uprightwood', axis=1)

In [16]:
preds.shape, truths.shape

((2516, 4), (1547, 6))

In [17]:
preds.label.value_counts()

2    2253
1     263
Name: label, dtype: int64

In [18]:
dis_truths = truths.dissolve(by='layer')
dis_preds = preds.dissolve(by='layer')

In [19]:
poly_IoU(dis_truths, dis_preds)

layer
groundwood     0.496085
uprightwood    0.503708
dtype: float64

In [20]:
deadwood_categories = [{'supercategory': 'deadwood', 'id':1, 'name':'uprightwood'},
                 
                       {'supercategory': 'deadwood', 'id':2, 'name':'groundwood'}]

raw_coco_eval = GisCOCOeval(raw_res_path, raw_res_path, 
                            None, None, deadwood_categories)

In [21]:
raw_coco_eval.prepare_data(gt_label_col='layer')

0it [00:00, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

In [22]:
raw_coco_eval.prepare_eval()

loading annotations into memory...
Done (t=0.03s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.02s)
creating index...
index created!


In [23]:
raw_coco_eval.coco_eval.params.maxDets = [1000, 10000]

In [24]:
raw_coco_eval.evaluate()


Evaluating for category uprightwood
Running per image evaluation...
Evaluate annotation type *segm*
DONE (t=0.44s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=10000 ] = 0.251
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=10000 ] = 0.507
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=10000 ] = 0.240
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=10000 ] = 0.164
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=10000 ] = 0.300
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=10000 ] = 0.900
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.346
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.275
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.386
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.900

Evaluating for 

## Half patch overlap and edge filtering

For this postprocessing method, mosaics are tiled so that the sliding window moves half tile lenght. For example, when moving row-wise, the first bottom-left coordinates are (0,0), and next ones (256,0), (512,0)... and same  is done column-wise. We discard all predicted polygons whose centroid point is not within the half-overlap area. For instance, for first tile (bottom left (0,0)), the x-coordinate must be between 128 and 384, for second tile (256,0) between 384 and 640, and likewise for y-coordinates.

The images used for predictions are buffered so that the whole area is covered, considering the discarding process.

In [25]:
pred_outpath = Path('../results/hp_overlap_filter/')
if not os.path.exists(pred_outpath):
    shutil.copytree('../results/template_folder/', pred_outpath)

In [ ]:
raw_path = Path('../../data/raw/hiidenportti/virtual_plots/buffered_test/images')
test_rasters = [raw_path/f for f in os.listdir(raw_path) if f.endswith('tif')]

for t in test_rasters:
    outfile_name = pred_outpath/f'raw_preds/{str(t).split("/")[-1][:-4]}.geojson'
    predict_instance_masks(path_to_model_config='../models/hiidenportti/mask_rcnn_X_101_32x8d_FPN_3x/config.yaml', 
                           path_to_image=str(t),
                           outfile=str(outfile_name),
                           processing_dir='temp',
                           tile_size=512,
                           tile_overlap=256,
                           smooth_preds=False,
                           use_tta=True,
                           coco_set='../../data/processed/hiidenportti/hiidenportti_valid.json',
                           postproc_results=True)

Modify as previously.

In [156]:
hp_res_path = Path('../results/hp_overlap_filter/')
truth_shps = sorted([hp_res_path/'vector_tiles'/f for f in os.listdir(hp_res_path/'vector_tiles')])
hp_raw_shps = sorted([hp_res_path/'raw_preds'/f for f in os.listdir(hp_res_path/'raw_preds')])
rasters = sorted([hp_res_path/'raster_tiles'/f for f in os.listdir(hp_res_path/'raster_tiles')])

In [157]:
for p, t in zip(hp_raw_shps, truth_shps):
    temp_pred = gpd.read_file(p)
    temp_truth = gpd.read_file(t)
    temp_pred['geometry'] = temp_pred.apply(lambda row: fix_multipolys(row.geometry) 
                                            if row.geometry.type == 'MultiPolygon' 
                                            else shapely.geometry.Polygon(row.geometry.exterior), axis=1)
    temp_pred['label'] += 1
    temp_pred = gpd.clip(temp_pred, box(*temp_truth.total_bounds))
    temp_pred = temp_pred[temp_pred.geometry.area > 16*0.04**2]
    temp_pred.to_file(hp_res_path/'predicted_vectors'/p.name)

In [158]:
pred_shps = sorted([hp_res_path/'predicted_vectors'/f for f in os.listdir(hp_res_path/'predicted_vectors')])

Collate all predictions into single dataframes

In [159]:
truths = None
preds = None

for p, t in zip(pred_shps, truth_shps):
    temp_pred = gpd.read_file(p)
    temp_truth = gpd.read_file(t)
    if truths is None:
        truths = temp_truth
        preds = temp_pred
    else:
        truths = pd.concat((truths, temp_truth))
        preds = pd.concat((preds, temp_pred))

In [160]:
preds['layer'] = preds.apply(lambda row: 'groundwood' if row.label == 2 else 'uprightwood', axis=1)

In [161]:
preds.layer.value_counts()

groundwood     2135
uprightwood     265
Name: layer, dtype: int64

In [34]:
dis_truths = truths.dissolve(by='layer')
dis_preds = preds.dissolve(by='layer')

In [35]:
poly_IoU(dis_truths, dis_preds)

layer
groundwood     0.513201
uprightwood    0.551898
dtype: float64

In [41]:
deadwood_categories = [{'supercategory': 'deadwood', 'id':1, 'name':'uprightwood'},
                 
                       {'supercategory': 'deadwood', 'id':2, 'name':'groundwood'}]

hp_coco_eval = GisCOCOeval(hp_res_path, hp_res_path, 
                            None, None, deadwood_categories)

In [42]:
hp_coco_eval.prepare_data(gt_label_col='layer')

0it [00:00, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

In [43]:
hp_coco_eval.prepare_eval()

loading annotations into memory...
Done (t=0.03s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.02s)
creating index...
index created!


In [44]:
hp_coco_eval.coco_eval.params.maxDets = [1000, 10000]

In [45]:
hp_coco_eval.evaluate()


Evaluating for category uprightwood
Running per image evaluation...
Evaluate annotation type *segm*
DONE (t=0.43s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=10000 ] = 0.336
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=10000 ] = 0.614
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=10000 ] = 0.367
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=10000 ] = 0.208
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=10000 ] = 0.409
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=10000 ] = 0.900
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.422
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.302
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.491
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.900

Evaluating for 

## Overlap, edge filtering and mask merging

Mask merging is built on previous predictions. In this step, for each polygon we check whether the ratio between intersection with any other polygon of the same class and the area of the polygon is more than 0.2. If yes, the polygon is merged to the other polygon with which it had intersection-over-area ratio.

In [50]:
merge_outpath = Path('../results/hp_merge/')
if not os.path.exists(merge_outpath):
    shutil.copytree('../results/template_folder/', merge_outpath)

Two iterations of merging is usually enough.

In [ ]:
for r in pred_shps:
    gdf_temp = gpd.read_file(r)
    standing = gdf_temp[gdf_temp.label==1].copy()
    fallen = gdf_temp[gdf_temp.label==2].copy()
    standing = merge_polys(standing, 0.2)
    fallen = merge_polys(fallen, 0.2)
    standing = merge_polys(standing, 0.2)
    fallen = merge_polys(fallen, 0.2)
    gdf_merged = pd.concat((standing, fallen))
    gdf_merged.to_file(merge_outpath/'predicted_vectors'/r.name, driver='GeoJSON')
    gdf_merged = None
    gdf_temp = None

In [56]:
merged_coco_eval = GisCOCOeval(merge_outpath, merge_outpath, None, None, deadwood_categories)
merged_coco_eval.prepare_data(gt_label_col='layer')
merged_coco_eval.prepare_eval()
merged_coco_eval.evaluate()

0it [00:00, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

loading annotations into memory...
Done (t=0.03s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.02s)
creating index...
index created!

Evaluating for category uprightwood
Running per image evaluation...
Evaluate annotation type *segm*
DONE (t=0.41s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.331
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.605
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.362
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.206
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.403
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.900
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.399
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.278
 Average Recall     (A

This usually worsens the results a bit, but the produced results are better suited for deriving forest charasteristics.

In [62]:
preds.reset_index(drop=True, inplace=True)
standing = merge_polys(preds[preds.label == 1].copy(), 0.2)
fallen = merge_polys(preds[preds.label == 2].copy(), 0.2)
standing = merge_polys(standing, 0.2)
fallen = merge_polys(fallen, 0.2)
preds_merged = pd.concat((standing, fallen))

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [63]:
preds_merged['layer'] = preds_merged.apply(lambda row: 'groundwood' if row.label == 2 else 'uprightwood', axis=1)
preds_merged.layer.value_counts()

groundwood     1665
uprightwood     258
Name: layer, dtype: int64

As the postprocessing merges data instead of dropping less certain predictions, total area and IoU remain the same as in previous step.

In [ ]:
preds_merged.to_file('../results/hiidenportti/merged_all.geojson')

# Full Evo dataset

Running predictions for Evo dataset takes so much time that it has been done separately. 

## No overlap, no post-processing

In [97]:
spk_raw_res_path = Path('../results/spk_raw/')
truth_shps = sorted([spk_raw_res_path/'vector_tiles'/f for f in os.listdir(spk_raw_res_path/'vector_tiles')])
spk_raw_shps = sorted([spk_raw_res_path/'raw_preds'/f for f in os.listdir(spk_raw_res_path/'raw_preds')])
rasters = sorted([spk_raw_res_path/'raster_tiles'/f for f in os.listdir(spk_raw_res_path/'raster_tiles')])

In [98]:
for p, t in zip(spk_raw_shps, truth_shps):
    temp_pred = gpd.read_file(p)
    temp_truth = gpd.read_file(t)
    temp_pred['geometry'] = temp_pred.apply(lambda row: fix_multipolys(row.geometry) 
                                            if row.geometry.type == 'MultiPolygon' 
                                            else shapely.geometry.Polygon(row.geometry.exterior), axis=1)
    temp_pred['label'] += 1
    temp_pred = gpd.clip(temp_pred, box(*temp_truth.total_bounds))
    temp_pred = temp_pred[temp_pred.geometry.area > 16*0.0485**2]
    temp_pred.to_file(spk_raw_res_path/'predicted_vectors'/p.name)

In [99]:
pred_shps = sorted([spk_raw_res_path/'predicted_vectors'/f for f in os.listdir(spk_raw_res_path/'predicted_vectors')])

In [100]:
truths = None
preds = None

for p, t in zip(pred_shps, truth_shps):
    temp_pred = gpd.read_file(p)
    temp_truth = gpd.read_file(t)
    if truths is None:
        truths = temp_truth
        preds = temp_pred
    else:
        truths = pd.concat((truths, temp_truth))
        preds = pd.concat((preds, temp_pred))

In [101]:
preds['layer'] = preds.apply(lambda row: 'groundwood' if row.label == 2 else 'uprightwood', axis=1)

In [102]:
preds.shape, truths.shape

((7017, 4), (5334, 4))

In [103]:
preds.layer.value_counts()

groundwood     5663
uprightwood    1354
Name: layer, dtype: int64

In [104]:
truths.rename(columns={'label':'layer'}, inplace=True)

In [105]:
dis_truths = truths.dissolve(by='layer')
dis_preds = preds.dissolve(by='layer')

In [106]:
poly_IoU(dis_truths, dis_preds)

layer
groundwood     0.453250
uprightwood    0.585433
dtype: float64

In [107]:
deadwood_categories = [{'supercategory': 'deadwood', 'id':1, 'name':'uprightwood'},
                 
                       {'supercategory': 'deadwood', 'id':2, 'name':'groundwood'}]

spk_raw_coco_eval = GisCOCOeval(spk_raw_res_path, spk_raw_res_path, 
                           None, None, deadwood_categories)
spk_raw_coco_eval.prepare_data(gt_label_col='label')
spk_raw_coco_eval.prepare_eval()
spk_raw_coco_eval.coco_eval.params.maxDets = [1000, 10000]
spk_raw_coco_eval.evaluate()

0it [00:00, ?it/s]

  0%|          | 0/71 [00:00<?, ?it/s]

loading annotations into memory...
Done (t=0.19s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.06s)
creating index...
index created!

Evaluating for category uprightwood
Running per image evaluation...
Evaluate annotation type *segm*
DONE (t=1.04s).
Accumulating evaluation results...
DONE (t=0.02s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=10000 ] = 0.259
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=10000 ] = 0.515
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=10000 ] = 0.241
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=10000 ] = 0.096
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=10000 ] = 0.346
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=10000 ] = 0.539
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.347
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.191
 Average Recal

## Half patch overlap and edge filtering

In [135]:
spk_res_path = Path('../results/spk_buffered/')
truth_shps = sorted([spk_res_path/'vector_tiles'/f for f in os.listdir(spk_res_path/'vector_tiles')])
spk_buf_raw_shps = sorted([spk_res_path/'raw_preds'/f for f in os.listdir(spk_res_path/'raw_preds')])
rasters = sorted([spk_res_path/'raster_tiles'/f for f in os.listdir(spk_res_path/'raster_tiles')])

In [136]:
for p, t in zip(spk_buf_raw_shps, truth_shps):
    temp_pred = gpd.read_file(p)
    temp_truth = gpd.read_file(t)
    temp_pred = gpd.clip(temp_pred, box(*temp_truth.total_bounds))
    temp_pred['geometry'] = temp_pred.apply(lambda row: fix_multipolys(row.geometry) 
                                            if row.geometry.type == 'MultiPolygon' 
                                            else shapely.geometry.Polygon(row.geometry.exterior), axis=1)
    temp_pred['label'] += 1
    temp_pred = temp_pred[temp_pred.geometry.area > 16*0.0485**2]
    temp_pred.to_file(spk_res_path/'predicted_vectors'/p.name)

In [137]:
pred_shps = sorted([spk_res_path/'predicted_vectors'/f for f in os.listdir(spk_res_path/'predicted_vectors')])

In [138]:
truths = None
preds = None

for p, t in zip(pred_shps, truth_shps):
    temp_pred = gpd.read_file(p)
    temp_truth = gpd.read_file(t)
    if truths is None:
        truths = temp_truth
        preds = temp_pred
    else:
        truths = pd.concat((truths, temp_truth))
        preds = pd.concat((preds, temp_pred))

In [139]:
preds['layer'] = preds.apply(lambda row: 'groundwood' if row.label == 2 else 'uprightwood', axis=1)

In [140]:
preds.shape, truths.shape

((7157, 4), (5334, 4))

In [141]:
preds.layer.value_counts()

groundwood     5777
uprightwood    1380
Name: layer, dtype: int64

In [142]:
truths.rename(columns={'label':'layer'}, inplace=True)

In [143]:
dis_truths = truths.dissolve(by='layer')
dis_preds = preds.dissolve(by='layer')

In [144]:
poly_IoU(dis_truths, dis_preds)

layer
groundwood     0.458800
uprightwood    0.598393
dtype: float64

In [145]:
deadwood_categories = [{'supercategory': 'deadwood', 'id':1, 'name':'uprightwood'},
                 
                       {'supercategory': 'deadwood', 'id':2, 'name':'groundwood'}]

spk_coco_eval = GisCOCOeval(spk_res_path, spk_res_path, 
                           None, None, deadwood_categories)
spk_coco_eval.prepare_data(gt_label_col='label')
spk_coco_eval.prepare_eval()
spk_coco_eval.coco_eval.params.maxDets = [1000, 10000]
spk_coco_eval.evaluate()

0it [00:00, ?it/s]

  0%|          | 0/71 [00:00<?, ?it/s]

loading annotations into memory...
Done (t=0.18s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.06s)
creating index...
index created!

Evaluating for category uprightwood
Running per image evaluation...
Evaluate annotation type *segm*
DONE (t=1.05s).
Accumulating evaluation results...
DONE (t=0.02s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=10000 ] = 0.330
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=10000 ] = 0.591
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=10000 ] = 0.352
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=10000 ] = 0.125
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=10000 ] = 0.449
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=10000 ] = 0.419
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.408
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.223
 Average Recal

## Overlap, edge filtering and mask merging

In [146]:
merge_outpath = Path('../results/spk_merge/')
if not os.path.exists(merge_outpath):
    shutil.copytree('../results/spk_template/', merge_outpath)

Two iterations of merging is usually enough.

In [ ]:
for r in pred_shps:
    gdf_temp = gpd.read_file(r)
    standing = gdf_temp[gdf_temp.label==1].copy()
    fallen = gdf_temp[gdf_temp.label==2].copy()
    if len(standing) > 0:
        standing = merge_polys(standing, 0.2)
        standing = merge_polys(standing, 0.2)
    if len(fallen) > 0:
        fallen = merge_polys(fallen, 0.2)
        fallen = merge_polys(fallen, 0.2)
    gdf_merged = pd.concat((standing, fallen))
    gdf_merged.to_file(merge_outpath/'predicted_vectors'/r.name, driver='GeoJSON')
    gdf_merged = None
    gdf_temp = None

In [152]:
merged_coco_eval = GisCOCOeval(merge_outpath, merge_outpath, None, None, deadwood_categories)
merged_coco_eval.prepare_data(gt_label_col='label')
merged_coco_eval.prepare_eval()
merged_coco_eval.evaluate()

0it [00:00, ?it/s]

  0%|          | 0/71 [00:00<?, ?it/s]

loading annotations into memory...
Done (t=0.18s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.05s)
creating index...
index created!

Evaluating for category uprightwood
Running per image evaluation...
Evaluate annotation type *segm*
DONE (t=1.00s).
Accumulating evaluation results...
DONE (t=0.02s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.319
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.571
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.342
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.123
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.431
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.521
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.391
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.219
 Average Recall     (A

In [153]:
preds.reset_index(drop=True, inplace=True)
standing = merge_polys(preds[preds.label == 1].copy(), 0.2)
fallen = merge_polys(preds[preds.label == 2].copy(), 0.2)
standing = merge_polys(standing, 0.2)
fallen = merge_polys(fallen, 0.2)
preds_merged = pd.concat((standing, fallen))

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [154]:
preds_merged['layer'] = preds_merged.apply(lambda row: 'groundwood' if row.label == 2 else 'uprightwood', axis=1)
preds_merged.layer.value_counts()

groundwood     4473
uprightwood    1312
Name: layer, dtype: int64

In [162]:
preds_merged.to_file('../results/sudenpesankangas/spk_merged.geojson')